In [ ]:
from kafka import KafkaConsumer
import json
try:
        # Create KafkaConsumer
        consumer = KafkaConsumer(
            "alerttopic",
            bootstrap_servers=['telemetry-kafka-external-bootstrap-observability-kafka.apps.zagaopenshift.zagaopensource.com:443'],
            value_deserializer=lambda v: v,  # Keep value as bytes for JSON decoding
            group_id="alert-producer",
            auto_offset_reset='earliest',
            consumer_timeout_ms=20000,
            security_protocol='SSL',  # Add SSL configuration if needed
            ssl_cafile='../telemetry_zagaopenshift.pem',
            ssl_certfile='../telemetry_zagaopenshift.crt'
        )
        print("Kafka consumer created successfully.")
except Exception as e:
        print(f"Error creating Kafka consumer: {e}")

In [ ]:
messages = []
try:
        # logger.info("Starting message consumption")
    for message in consumer:
            # logger.debug(f"Consumed message: {message.value}")
            json_message = json.loads(message.value.decode('utf-8'))  # Decode and parse JSON
            messages.append(json_message)
            if len(messages) > 0:
                print("Message consumed")
                consumer.close()
except KeyboardInterrupt:
        # logger.info("Stopping consumer...")
        print("Keyboard Interrupt")
except Exception as e:
        # logger.error(f"An error occurred: {e}")
        print(e)
finally:
        print(messages)

In [1]:
import asyncio
import json
import nest_asyncio
from kafka import KafkaConsumer
import websockets
from websockets.exceptions import ConnectionClosedError, ConnectionClosedOK

# Apply the nest_asyncio patch to allow nested event loops
nest_asyncio.apply()

# Kafka consumer configuration
def create_kafka_consumer():
    return KafkaConsumer(
        "alerttopic",
        bootstrap_servers=['telemetry-kafka-external-bootstrap-observability-kafka.apps.zagaopenshift.zagaopensource.com:443'],
        value_deserializer=lambda v: json.loads(v.decode('utf-8')),
        group_id="alert-producer",
        auto_offset_reset='earliest',
        consumer_timeout_ms=1000,
        security_protocol='SSL',
        ssl_cafile='../telemetry_zagaopenshift.pem',
        ssl_certfile='../telemetry_zagaopenshift.crt'
    )

# List to keep track of connected WebSocket clients
connected_clients = set()

async def kafka_consumer_handler(consumer):
    try:
        while True:
            for message in consumer:
                alert_data = message.value
                print(f"Received message from Kafka: {alert_data}")
                await broadcast(alert_data)
            await asyncio.sleep(1)  # Sleep briefly to yield control back to the event loop
    except Exception as e:
        print(f"Error in Kafka consumer handler: {e}")

async def websocket_handler(websocket, path):
    # Register client
    connected_clients.add(websocket)
    print(f"New client connected: {websocket.remote_address}, Path: {path}")
    try:
        await websocket.send(json.dumps({"message": "Connected to alert notification service"}))
        async for message in websocket:
            # Here you can handle messages received from WebSocket clients if needed
            print(f"Received message from client: {message}")
    except (ConnectionClosedError, ConnectionClosedOK) as e:
        print(f"Client {websocket.remote_address} disconnected: {e}")
    except Exception as e:
        print(f"Error in websocket handler for client {websocket.remote_address}: {e}")

async def broadcast(message):
    if connected_clients:
        message = json.dumps(message)
        await asyncio.gather(*[client.send(message) for client in connected_clients])

async def main():
    # Create Kafka consumer
    consumer = create_kafka_consumer()
    
    # Start Kafka consumer handler
    consumer_task = asyncio.create_task(kafka_consumer_handler(consumer))

    # Start WebSocket server
    async with websockets.serve(websocket_handler, "localhost", 6789):
        await asyncio.Future()  # run forever

    # Wait for the consumer task to complete (which it shouldn't)
    await consumer_task

# Entry point
if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.create_task(main())
    try:
        loop.run_forever()
    except KeyboardInterrupt:
        print("Server stopped by user")


Server stopped by user


In [2]:
import asyncio
import websockets

async def connect():
    async with websockets.connect("ws://localhost:6789") as websocket:
        print("Connected to server")
        await websocket.send("Hello, Server!")
        response = await websocket.recv()
        print(f"Received response: {response}")

asyncio.run(connect())

New client connected: ('::1', 59630, 0, 0), Path: /
Received message from client: Hello, Server!


Connected to server
Received response: {"message": "Connected to alert notification service"}
